In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, desc, lit
import glob

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("WordCount") \
    .getOrCreate()

23/06/19 08:57:11 WARN Utils: Your hostname, DESKTOP-L5DLPTK resolves to a loopback address: 127.0.1.1; using 192.168.144.224 instead (on interface wifi0)
23/06/19 08:57:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/19 08:57:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, desc


In [20]:
file_paths = glob.glob('/home/lucas/Projeto2Savino/src/clean_data/*json')

In [41]:
def count_words(file_path, top_n_words=30):
    """
    Conta a frequência de palavras de um arquivo
    json e seleciona as top_n_wordsmais frequentes.
    """
    file_name = file_path.split('/')[-1].split('.')[0]
    df = spark.read.json('/home/lucas/Projeto2Savino/src/clean_data/CanalTechLimpo.json')
    df_words = df.withColumn("words", split(df.text, "\s+"))
    df_exploded = df_words.select(df_words.link, explode(df_words.words).alias("word"))
    word_counts = df_exploded.groupBy("word").count()
    word_counts_sorted = word_counts.orderBy(word_counts["count"].desc()).limit(top_n_words)
    site_word_counts = word_counts_sorted.withColumn("site", lit(file_name))
    return site_word_counts

In [44]:
dfs = []
for file_ in file_paths:
    word_counts = count_words(file_)
    dfs.append(word_counts)

Unindo todos os DataFrames em um só

In [45]:
result_df = dfs[0]
for i in range(1, len(dfs)):
    result_df = result_df.union(dfs[i])

In [47]:
result_df.show()

+------------+-----+--------------+
|        word|count|          site|
+------------+-----+--------------+
|     AirPods|   84|CanalTechLimpo|
|       Apple|   50|CanalTechLimpo|
|         Pro|   45|CanalTechLimpo|
|           2|   28|CanalTechLimpo|
|      estojo|   22|CanalTechLimpo|
|      ouvido|   21|CanalTechLimpo|
|      iPhone|   20|CanalTechLimpo|
|carregamento|   20|CanalTechLimpo|
|       fones|   18|CanalTechLimpo|
|        fone|   17|CanalTechLimpo|
|     bateria|   16|CanalTechLimpo|
|        USBC|   14|CanalTechLimpo|
|        novo|   14|CanalTechLimpo|
|       Passo|   14|CanalTechLimpo|
|       Beats|   12|CanalTechLimpo|
|          TV|   12|CanalTechLimpo|
|      volume|   12|CanalTechLimpo|
|         Fit|   11|CanalTechLimpo|
|       áudio|   11|CanalTechLimpo|
|        deve|   11|CanalTechLimpo|
+------------+-----+--------------+
only showing top 20 rows



In [43]:
word_counts.show()

+------------+-----+--------------+
|        word|count|          site|
+------------+-----+--------------+
|     AirPods|   84|CanalTechLimpo|
|       Apple|   50|CanalTechLimpo|
|         Pro|   45|CanalTechLimpo|
|           2|   28|CanalTechLimpo|
|      estojo|   22|CanalTechLimpo|
|      ouvido|   21|CanalTechLimpo|
|      iPhone|   20|CanalTechLimpo|
|carregamento|   20|CanalTechLimpo|
|       fones|   18|CanalTechLimpo|
|        fone|   17|CanalTechLimpo|
|     bateria|   16|CanalTechLimpo|
|        USBC|   14|CanalTechLimpo|
|        novo|   14|CanalTechLimpo|
|       Passo|   14|CanalTechLimpo|
|       Beats|   12|CanalTechLimpo|
|          TV|   12|CanalTechLimpo|
|      volume|   12|CanalTechLimpo|
|         Fit|   11|CanalTechLimpo|
|        deve|   11|CanalTechLimpo|
|       áudio|   11|CanalTechLimpo|
+------------+-----+--------------+
only showing top 20 rows



In [23]:
df = spark.read.json('/home/lucas/Projeto2Savino/src/clean_data/CanalTechLimpo.json')

In [24]:
df.show()

+--------------------+--------------------+
|                link|                text|
+--------------------+--------------------+
|https://canaltech...|Após pressão Uniã...|
|https://canaltech...|Beats Fit Pro apo...|
|https://canaltech...|Apple prepara set...|
|https://canaltech...|Apple lançou AirP...|
|https://canaltech...|Apresentado final...|
|https://canaltech...|grandes qualidade...|
|https://canaltech...|AirPods Pro 2 lan...|
|https://canaltech...|AirPods excelente...|
+--------------------+--------------------+



In [25]:
df_words = df.withColumn("words", split(df.text, "\s+"))

In [26]:
df_exploded = df_words.select(df_words.link, explode(df_words.words).alias("word"))

In [29]:
word_counts = df_exploded.groupBy("word").count()

In [30]:
word_counts_sorted = word_counts.orderBy(word_counts["count"].desc()).limit(30)

In [32]:
word_counts_sorted.show()

+------------+-----+
|        word|count|
+------------+-----+
|     AirPods|   84|
|       Apple|   50|
|         Pro|   45|
|           2|   28|
|      estojo|   22|
|      ouvido|   21|
|      iPhone|   20|
|carregamento|   20|
|       fones|   18|
|        fone|   17|
|     bateria|   16|
|        USBC|   14|
|        novo|   14|
|       Passo|   14|
|       Beats|   12|
|          TV|   12|
|      volume|   12|
|         Fit|   11|
|        deve|   11|
|       áudio|   11|
+------------+-----+
only showing top 20 rows



In [15]:
word_counts_sorted.write.csv("/home/lucas/Projeto2Savino/src/word_counts/canal_tech.csv", header=True, mode="overwrite")